# Artificial Neural Network

### Importing The Libraries

In [60]:
import numpy as np 
import pandas as pd
import tensorflow as tf

In [61]:
tf.__version__

'2.9.1'

## Step - 1 Data Preprocessing

### Importing The Dataset

In [62]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[ : , -1].values

In [63]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [64]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding Categorical Data

Label Encoding "Gender" Column

In [65]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[: , 2] = le.fit_transform(x[:,2])

In [66]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" Column

In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [68]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting The Dataset Into the Training set and Test Set

In [69]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling 

In [70]:
from sklearn.preprocessing import StandardScaler   #to apply standardization
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Part 2 - Building the ANN

Initializing ANN

In [71]:
ann = tf.keras.models.Sequential()

Adding the Input layer and the First Hidden Layer

In [72]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Adding The Second Hidden Layer

In [73]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Adding the Output Layer

In [74]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### Part 3 - Training the ANN

Compiling the ANN

In [75]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Training the ANN on the Training Set

In [76]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 5s 9ms/step - loss: 0.5522 - accuracy: 0.7745
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.4700 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4446 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4313 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4243 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4184 - accuracy: 0.7976
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4103 - accuracy: 0.8204
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3991 - accuracy: 0.8301
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3858 - accuracy: 0.8393
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3738 - accura

## Part 4 - Making The Predictions and Evaluating the Model

Predicting the Result of a Single Observation

Geography = France<br> 
Credit Score = 600<br>
Gender = Male <br>
Age = 40<br>
Tenure = 3<br>
Balance = 60000<br>
Number Of Products = 2<br>
Does this customer have a credit card ? yes<br>
Is this customer an Active Member Yes<br>
Estimated Salary = 50000<br>
So, shouuld we say goodbye to the Customer?

In [81]:
x_temp = [1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]
print(ann.predict(sc.transform([x_temp])) > 0.5)

1/1 [==============================] - 0s 277ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

Predicting the Test set Results

In [82]:
y_pred = ann.predict(x_test)
y_pred = ( y_pred > 0.5 )
print(np.concatenate((y_pred.reshape(-1,1),y_test.reshape(-1,1)),1))

63/63 [==============================] - 1s 7ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the Confusion Matrix

In [85]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[1517   78]
 [ 201  204]]
0.8605
